In [306]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
from typing import List, Optional, Callable, Tuple
import random
import itertools

random.seed(3)

<font size = 6> Steady state genetic algo </font>

https://www.geeksforgeeks.org/steady-state-genetic-algorithm-ssga/

Parameters 
1. Param Grid: Given
2. Genome: One combination of Param Grid
3. Population: All combinations of Param Grid considered

Functions
1. Genetic Representation of Genome, Population (solutions)
2. Generating new solutions
3. Fitness Value functions: MSE, MAE, etc
4. Elitism/Selection Function: Choose which Genome to undergo CrossOver, can use python's choice function, weight = Fitness Value
5. Cross over Function
6. Mutation Algo: Usually mutation rate is set to 1/L, where L is the length of the bitstring.
7. Evolution Algo

Classes
1. Genome
2. Population
3. GenomSearch
    - GenomSearchCV

In [307]:
"""
Input example:

params_grid = {
    'n': {range: [low, high], dtype: int}   ## (Low, High) represents the range, dtype = int, float etc
    'objective': (cat: [cat1, cat2, cat3,...], dtype: category }
    'gamma': {range: [low, high], dtype: float}
}

available dtypes:
category, int, float

population = [Genome1, Genome2, Genome3, ...]


need to make one for pipelining so like "PCA__n componenes"
"""

'\nInput example:\n\nparams_grid = {\n    \'n\': {range: [low, high], dtype: int}   ## (Low, High) represents the range, dtype = int, float etc\n    \'objective\': (cat: [cat1, cat2, cat3,...], dtype: category }\n    \'gamma\': {range: [low, high], dtype: float}\n}\n\navailable dtypes:\ncategory, int, float\n\npopulation = [Genome1, Genome2, Genome3, ...]\n\n\nneed to make one for pipelining so like "PCA__n componenes"\n'

In [308]:
"""Helpful functions:
1. Timsort, insertion, merge
2. Merge dictionaries
"""


MIN_MERGE = 32


def calcMinRun(n):
    """Returns the minimum length of a
    run from 23 - 64 so that
    the len(array)/minrun is less than or
    equal to a power of 2.

    e.g. 1=>1, ..., 63=>63, 64=>32, 65=>33,
    ..., 127=>64, 128=>32, ...
    """
    r = 0
    while n >= MIN_MERGE:
        r |= n & 1
        n >>= 1
    return n + r


# This function sorts array from left index to
# to right index which is of size atmost RUN
def insertionSort(pop, left, right, dir):
    for i in range(left + 1, right + 1):
        j = i

        if dir == 1:       ## Ascendimg
            while j > left and pop[j] < pop[j - 1]:
                pop[j], pop[j - 1] = pop[j - 1], pop[j]
                j -= 1
        else:               ## Descending
            while j > left and pop[j] > pop[j - 1]:
                pop[j], pop[j - 1] = pop[j - 1], pop[j]
                j -= 1


# Merge function merges the sorted runs
def merge(pop, l, m, r, dir = 1):

    # original array is broken in two parts
    # left and right array
    len1, len2 = m - l + 1, r - m
    left, right = [], []
    for i in range(0, len1):
        left.append(pop[l + i])
    for i in range(0, len2):
        right.append(pop[m + 1 + i])

    i, j, k = 0, 0, l

    # after comparing, we merge those two array
    # in larger sub array
    if dir == 1:   ## Ascending
        while i < len1 and j < len2:
            if left[i] <= right[j]:
                pop[k] = left[i]
                i += 1

            else:
                pop[k] = right[j]
                j += 1

            k += 1
    else:           ## Descending
        while i < len1 and j < len2:
            if left[i] >= right[j]:
                pop[k] = left[i]
                i += 1

            else:
                pop[k] = right[j]
                j += 1

            k += 1

    # Copy remaining elements of left, if any
    while i < len1:
        pop[k] = left[i]
        k += 1
        i += 1

    # Copy remaining element of right, if any
    while j < len2:
        pop[k] = right[j]
        k += 1
        j += 1

# Iterative Timsort function to sort the
# array[0...n-1] (similar to merge sort)
def tim_sort(pop, dir = 1):
    """
    Purpose:
    Sorts population based on fitness values

    Input: 
    pop: Takes in a population
    dir: '1' - Ascending, '0' - 'Descending'
    """
    n = len(pop)
    minRun = calcMinRun(n)

    # Sort individual subarrays of size RUN
    for start in range(0, n, minRun):
        end = min(start + minRun - 1, n - 1)
        insertionSort(pop, start, end, dir)

    # Start merging from size RUN (or 32). It will merge
    # to form size 64, then 128, 256 and so on ....
    size = minRun
    while size < n:

        # Pick starting point of left sub array. We
        # are going to merge arr[left..left+size-1]
        # and arr[left+size, left+2*size-1]
        # After every merge, we increase left by 2*size
        for left in range(0, n, 2 * size):

            # Find ending point of left sub array
            # mid+1 is starting point of right sub array
            mid = min(n - 1, left + size - 1)
            right = min((left + 2 * size - 1), (n - 1))

            # Merge sub array arr[left.....mid] &
            # arr[mid+1....right]
            if mid < right:
                merge(pop, left, mid, right)

        size = 2 * size

    return pop

def merge_dicts(dict1, dict2):
    # Check for overlapping keys
    intersection = set(dict1.keys()) & set(dict2.keys())
    if intersection:
        raise ValueError(f"Overlapping keys found: {intersection}")

    # Merge the dictionaries
    merged_dict = dict1.copy()
    merged_dict.update(dict2)
    
    return merged_dict



In [313]:
class Genome():
    def __init__(self, params):
        self.params = params
        self.fitness = self.fitness(params)

    ## Comparing operators
    def __lt__(self, other):
        # Define custom behavior for "<"
        return self.fitness < other.fitness

    def __le__(self, other):
        # Define custom behavior for "<="
        return self.fitness <= other.fitness

    def __ge__(self, other):
        # Define custom behavior for ">="
        return self.fitness >= other.fitness

    def __eq__(self, other):
        # Define custom behavior for "=="
        return self.fitness == other.fitness

    def __ne__(self, other):
        # Define custom behavior for "!="
        return self.fitness != other.fitness
    
    def __len__(self):
        return len(self.params.keys())
    
    def __repr__(self):
        string = f'Genome = Params : {self.params}, Fitness : {self.fitness}'
        return string
    
    def extract(self, start, stop = None, step = 1):
        """
        Purpose:
        Just for ease of slicing
        
        Parameters:
        key: Can be slice object
        """
        if not stop: ## If stop == None, means only want extract one
            return dict([list(self.params.items())[start]])
            
        elif stop > start and stop <= len(self):
            target = dict([list(self.params.items())[i] for i in range(start, stop, step)])
            return target
        else:
            raise IndexError(f"Index out of range.")

    def get_params(self):
        return self.params

    def get_fitness(self):
        return self.fitness


    
    def fitness(self, params):
        """
        Input:
        Takes in 1 genome/set of params
        """
        sum = 0
        for key, item in params.items():  
            try:
                sum += item
            except:
                sum += ord(item)
        
        return sum
    

        

class Population():
    def __init__(self, param_grid, n):
        self.choices = param_grid
        self.n = n
        self.population = self.generate_pop()

    def __getitem__(self, key):
        if isinstance(key, int):
            return self.population[key]
        
        elif isinstance(key, slice):
            start, stop, step = key.indices(self.n)
            target = [self.population[i] for i in range(start, stop, step)]
            return target
        else:
            raise TypeError(f"Invalid Argument type. Must be int or slice but {type(key)} given.")
        
    def __repr__(self):
        string = ""
        for genome in self.population:
            string += f'{genome}\n'
        return string
        

    def get_pop(self):
        if self.population:
            return self.population
        else:
            print("No Population Generated Yet!")
            return


    def get_fit(self, n):
        """
        Purpose: 
        Shortcut to get fitness
        
        Parameters
        n: index of Genomesto get the fitness vals
         
        Returns:
        Fitness value
        """
        return self[n].get_fitness()
    


    def generate_pop(self):
        
        """
        Purpose:
        To initialize population

        Parameters
        n: Number of Genomes

        Returns: None

        """
        pop = []
        for i in range(self.n):
            params = {}
            for key, items in self.choices.items():
                if items['dtype'] == 'category':
                    params[key] = random.choice(items['cat'])
                elif items['dtype'] == 'int':
                    low, high = items['range']
                    params[key] = random.randint(low, high)
                elif items['dtype'] == 'float':
                    params[key] = round(random.uniform(low, high), 5)
                else:
                    print('Error in {key} dtypes')
                    continue
            new_genome = Genome(params)
            pop += [new_genome]

        pop = tim_sort(pop, 0)

        return pop


    def elitism(self, n):
        """
        Purpose:
        To choose which n Genomes to bring over to next gen

        Parameters:
        n: Number of Genomes

        Returns:
        Top n Genomes
        """
        if not self.population:
            print("No Population to choose from")

        else:
            top_n = self[0:n]

        return top_n
    


    def n_cross_over(self, genes, n, verbose = True):
        """
        Purpose: 
        To cross over 2 genomes
        
        Parameters:
        genes: (a,b) Cross over genomes in pop[a] and pop[b]
        n: Number of points to cross over
        verbose: Illustrate parents to genomes
        
        Returns:
        2 Genomes, Crossed over
        """
        a,b = genes
        gene1 = self[a]
        gene2 = self[b]

        len1 = len(gene1)
        len2 = len(gene2)
        if len1 != len2:
            raise ValueError("Genomes must be of same length")
        
        if len1 < 2:
            raise Exception("Too short!")
        
        #########################
        idx = random.randint(1, len1-2)  ## Point chosen must be inbetween and cut-off AT key AT index
        
        gene11 = gene1.extract(0, idx)   ## Params from index = 0 to index = idx-1
        gene12 = gene1.extract(idx, len1)   ## not slicing genomes but params

        gene21 = gene2.extract(0, idx) 
        gene22 = gene2.extract(idx, len2)


        gene3 = merge_dicts(gene11, gene22)
        gene4 = merge_dicts(gene21, gene12)
        
        gene3 = Genome(gene3)
        gene4 = Genome(gene4)
        #########################



        choices = list(range(1,len1-2))

        idxs = tim_sort(np.random.choice(choices, size = n-1, replace = False))

        






        if verbose:
            print(f'------------Parent Genomes------------\n{gene1}\n{gene2}\n')
            print(f'After {n}-point(s) crossover at index : {idx}\n')
            print(f'------------Children Genomes-------------\n{gene3}\n{gene4}\n')

        return (gene3, gene4)


       

    def cross_over(self, points):
            ## use selection to select to 
            pass


    

In [314]:
## Test

param_grid = {
    'a': {'range': [1,10], 'dtype' : 'int'},
    'b': {'range': [1,10], 'dtype' : 'int'},
    'c': {'range': [1,10], 'dtype' : 'int'},
    'd': {'cat':['A','B','C'], 'dtype' : 'category'}
}

pop = Population(param_grid, 10)
pop

# pop.elitism(2)


Genome = Params : {'a': 10, 'b': 7, 'c': 10, 'd': 'A'}, Fitness : 92
Genome = Params : {'a': 7, 'b': 7, 'c': 10, 'd': 'B'}, Fitness : 90
Genome = Params : {'a': 7, 'b': 10, 'c': 6, 'd': 'C'}, Fitness : 90
Genome = Params : {'a': 7, 'b': 5, 'c': 7, 'd': 'C'}, Fitness : 86
Genome = Params : {'a': 3, 'b': 6, 'c': 9, 'd': 'C'}, Fitness : 85
Genome = Params : {'a': 10, 'b': 5, 'c': 5, 'd': 'A'}, Fitness : 85
Genome = Params : {'a': 10, 'b': 2, 'c': 4, 'd': 'C'}, Fitness : 83
Genome = Params : {'a': 3, 'b': 8, 'c': 4, 'd': 'B'}, Fitness : 81
Genome = Params : {'a': 6, 'b': 1, 'c': 5, 'd': 'C'}, Fitness : 79
Genome = Params : {'a': 3, 'b': 6, 'c': 2, 'd': 'A'}, Fitness : 76

In [315]:
pop[1].extract(1,4)

{'b': 7, 'c': 10, 'd': 'B'}

In [288]:
list(range(1,-1,1))

[]

In [264]:
tim_sort(np.random.choice([1,2,3,4,5,6,7,8,9,10], size = 3, replace = False))

array([ 2,  7, 10])

In [254]:
pop.n_cross_over((1,2),1)

------------Parent Genomes------------
Genome = Params : {'a': 7, 'b': 7, 'c': 10, 'd': 'B'}, Fitness : 90
Genome = Params : {'a': 7, 'b': 10, 'c': 6, 'd': 'C'}, Fitness : 90

After 1-point(s) crossover at index : 1

------------Children Genomes-------------
Genome = Params : {'a': 7, 'b': 10, 'c': 6, 'd': 'C'}, Fitness : 90
Genome = Params : {'a': 7, 'b': 7, 'c': 10, 'd': 'B'}, Fitness : 90



(Genome = Params : {'a': 7, 'b': 10, 'c': 6, 'd': 'C'}, Fitness : 90,
 Genome = Params : {'a': 7, 'b': 7, 'c': 10, 'd': 'B'}, Fitness : 90)